In [2]:
import dash
from dash import dcc, html
import pandas as pd
import plotly.express as px
from dash.dependencies import Input, Output

In [24]:
df = pd.read_csv('C:\\Users\\Ben\\OneDrive\\Documents\\MarylandData\\Maryland_DS_Projects_Github\\Maryland_Statewide_Vehicle_Crashes.csv')
# Assuming df is your DataFrame
df_new = df[['YEAR', 'LIGHT_DESC', 'COUNTY_DESC', 'REPORT_TYPE', 'WEATHER_DESC', 'ACC_DATE', 'ACC_TIME', 'LATITUDE', 'LONGITUDE', 'JUNCTION_DESC']]

C:\Users\Ben\AppData\Local\Temp\ipykernel_24348\1924590499.py:1: DtypeWarning: Columns (33,45) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('C:\\Users\\Ben\\OneDrive\\Documents\\MarylandData\\Maryland_DS_Projects_Github\\Maryland_Statewide_Vehicle_Crashes.csv')


In [25]:
df_new.head()
# Replace all single quotes in the 'COUNTY_DESC' column with an empty string
df_new['COUNTY_DESC'] = df_new['COUNTY_DESC'].str.replace("'", "")

df_new['COUNTY_DESC'].unique()

C:\Users\Ben\AppData\Local\Temp\ipykernel_24348\1612809616.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['COUNTY_DESC'] = df_new['COUNTY_DESC'].str.replace("'", "")


array(['Baltimore', 'Montgomery', 'Queen Annes', 'Baltimore City',
       'Frederick', 'Howard', 'Cecil', 'Anne Arundel', 'Worcester',
       'Dorchester', 'Harford', 'Washington', 'Charles', 'Prince Georges',
       'Talbot', 'Wicomico', 'Kent', 'St. Marys', 'Calvert', 'Caroline',
       'Carroll', 'Garrett', 'Allegany', 'Somerset', nan], dtype=object)

In [26]:
# List of counties you want to keep
counties_to_keep = ['Anne Arundel', 'Prince Georges', 'Montgomery', 
                    'Howard','Queen Annes']

# Filter the dataframe to include only the rows where County is in counties_to_keep
df_new = df_new[df_new['COUNTY_DESC'].isin(counties_to_keep)]
print(df_new['COUNTY_DESC'].unique())

['Montgomery' 'Queen Annes' 'Howard' 'Anne Arundel' 'Prince Georges']


In [27]:
df_new.to_csv('DF_output.csv', index=False)

In [83]:
#Should change this to Input - but should reference other code to figure out what Output is
df = pd.read_csv('DF_output.csv')

# Convert ACC_DATE to datetime format
df['ACC_DATE'] = pd.to_datetime(df['ACC_DATE'], format='%Y%m%d')

# Combine ACC_DATE and ACC_TIME into a single datetime column
df['ACC_DATETIME'] = pd.to_datetime(df['ACC_DATE'].astype(str) + ' ' + df['ACC_TIME'])

# Extract hour and month from ACC_DATETIME
df['HOUR'] = df['ACC_DATETIME'].dt.hour
df['MONTH'] = df['ACC_DATETIME'].dt.month
df['DAY'] = df['ACC_DATETIME'].dt.dayofweek

In [84]:
# Pre-compute the data for the plots
hourly_counts = df.groupby(['HOUR', 'COUNTY_DESC', 'REPORT_TYPE']).size().reset_index()
hourly_counts.columns = ['HOUR', 'COUNTY_DESC', 'REPORT_TYPE', 'COUNT']
county_counts = df.groupby(['COUNTY_DESC', 'REPORT_TYPE']).size().unstack(fill_value=0).reset_index()

#For Long Term Trends
daily_counts = df.groupby(['ACC_DATE', 'COUNTY_DESC']).size().reset_index()
daily_counts['ACC_DATE'] = pd.to_datetime(daily_counts['ACC_DATE'])
daily_counts.columns = ['ACC_DATE', 'COUNTY_DESC', 'COUNT']
daily_counts.reset_index(inplace=True)
daily_counts['day_of_year'] = daily_counts['ACC_DATE'].dt.dayofyear
pre_covid = daily_counts[daily_counts['ACC_DATE'] < '2020-03-1']
post_covid = daily_counts[daily_counts['ACC_DATE'] >= '2020-03-1']
pre_covid_avg = pre_covid.groupby('day_of_year')['COUNT'].mean()
post_covid = post_covid.merge(pre_covid_avg, on='day_of_year', how='left', suffixes=('_post', '_pre'))
post_covid['percentage_of_pre_covid'] = (post_covid['COUNT_post'] / post_covid['COUNT_pre']) * 100
post_covid['Day_of_Year'] = post_covid['ACC_DATE'].dt.dayofyear
post_covid['Year'] = post_covid['ACC_DATE'].dt.year
post_covid['percentage_of_pre_covid_smooth'] = post_covid['percentage_of_pre_covid'].rolling(window=30).mean()
df.to_csv('DF_Output_Processed.csv', index=False)

# Pre-compute the data for the plots
monthly_counts = df.groupby(['MONTH', 'COUNTY_DESC', 'REPORT_TYPE']).size().reset_index()
monthly_counts.columns = ['MONTH', 'COUNTY_DESC', 'REPORT_TYPE', 'COUNT']

# Pre-compute the data for the plots
daily_counts = df.groupby(['DAY', 'COUNTY_DESC', 'REPORT_TYPE']).size().reset_index()
daily_counts.columns = ['DAY', 'COUNTY_DESC', 'REPORT_TYPE', 'COUNT']

# In FeatureEngineering.ipynb
hourly_counts.to_csv('Assets/hourly_counts.csv', index=False)
post_covid.to_csv('Assets/post_covid.csv', index=False)
county_counts.to_csv('Assets/county_counts.csv', index=False)
monthly_counts.to_csv('Assets/monthly_counts.csv', index=False)
daily_counts.to_csv('Assets/daily_counts.csv', index=False)
